In [ ]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from scipy.stats import uniform, randint
import numpy as np
import joblib
from sklearn.preprocessing import LabelEncoder
import anndata as ad
import pandas as pd
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import xgboost as xgb
import scanpy as sc

# 01 - Load model

In [ ]:
import pickle  # or use joblib

adata=ad.read_h5ad('/home/local.hcpa.ufrgs.br/tkruger/V02_Glioblastoma_atlas/adatas/adata.h5ad')

with open("final_xgb_model_20250504_025832.pkl", "rb") as f:
    model = pickle.load(f)

importance = model.feature_importances_
feature_names = adata.var_names

importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importance
}).sort_values(by='Importance', ascending=False)

importance_df

adatab=ad.read_h5ad('/home/local.hcpa.ufrgs.br/tkruger/V02_Glioblastoma_atlas/adatas/adata_5.h5ad')

#Make row names unique
adata.obs_names_make_unique()
adatab.obs_names_make_unique()

#Find intersections between adatas
match = np.intersect1d(adata.obs_names, adatab.obs_names)

#Keep adata matrix with obs that are in adatab
adata = adata[match]

#Map classes from adatab to adata
adata.obs['broad_cell_type'] = adata.obs.index.map(adatab.obs['broad_cell_type'])

In [ ]:
importance_df

In [ ]:
adata.obsm["X_umap"] = adatab.obsm["X_umap"].copy()

In [ ]:
sc.pl.umap(adatab, color='NPC2')

# 02 - Remove half of the features

In [ ]:
remove = importance_df.tail(6556)['Feature']

In [ ]:
adata.shape

In [ ]:
adata = adata[:, ~adata.var_names.isin(remove)]

In [ ]:
adata.shape

In [ ]:
adata.obs

In [ ]:
#Extract adata matrix
X = adata.X

In [ ]:
if not isinstance(X, np.ndarray):
    X_array = X.toarray()
else:
    X_array = X

In [ ]:
#Extract classes
y = adata.obs['broad_cell_type']

In [ ]:
df = pd.DataFrame({'label': y.values})

In [ ]:
#Save matrices and labels of train and test
np.save('main_X_50.npy', X_array)
np.save('main_Y_50.npy', y)
df.to_csv('main_df_50.csv')

# 03 - Define Model

In [ ]:
# Best params:
best_learning_rate = 0.22959818254342154
best_max_depth = 7
best_n_estimators = 70

In [ ]:
#Define model
model = XGBClassifier(
    learning_rate=best_learning_rate,
    max_depth=best_max_depth,
    n_estimators=best_n_estimators,
    use_label_encoder=False,
    eval_metric='logloss',
    n_jobs=1,
    verbosity=1
)


# 04 - Data split

In [ ]:
#Divide dataset into train and test keeping proportions
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#Convert matrices to dense
X_train_dense = X_train.toarray()
X_test_dense = X_test.toarray()

In [ ]:
#Get rows sums
row_sums_train = X_train_dense.sum(axis=1, keepdims=True)
row_sums_test = X_test_dense.sum(axis=1, keepdims=True)

In [ ]:
#Normalize both matrices using row sums and scaling to 1000k counts
X_train_normalized = X_train_dense / row_sums_train * 1000
X_test_normalized = X_test_dense / row_sums_test * 1000

In [ ]:
#Apply log1p to normalized counts
X_train_log1p = np.log1p(X_train_normalized)
X_test_log1p = np.log1p(X_test_normalized)

In [ ]:
#Apply scaler for mean = 0
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_log1p)
X_test_scaled = scaler.fit_transform(X_test_log1p)

In [ ]:
#Save tuning subset
np.save('feature_reduction/01_X_train.npy',X_train_scaled)
np.save('feature_reduction/01_y_train.npy',y_train)
np.save('feature_reduction/01_X_test.npy',X_test_scaled)
np.save('feature_reduction/01_y_test.npy',y_test)

In [ ]:
# Keep all features

In [ ]:
X = adata.X
if not isinstance(X, np.ndarray):
    X_array = X.toarray()
else:
    X_array = X
y = adata.obs['broad_cell_type']
df = pd.DataFrame({'label': y.values})
np.save('main_X_50.npy', X_array)
np.save('main_Y_50.npy', y)
df.to_csv('main_df_50.csv')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_dense = X_train.toarray()
X_test_dense = X_test.toarray()
row_sums_train = X_train_dense.sum(axis=1, keepdims=True)
row_sums_test = X_test_dense.sum(axis=1, keepdims=True)
X_train_normalized = X_train_dense / row_sums_train * 1000
X_test_normalized = X_test_dense / row_sums_test * 1000
X_train_log1p = np.log1p(X_train_normalized)
X_test_log1p = np.log1p(X_test_normalized)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_log1p)
X_test_scaled = scaler.fit_transform(X_test_log1p)
np.save('feature_reduction/01_X_train.npy',X_train_scaled)
np.save('feature_reduction/01_y_train.npy',y_train)
np.save('feature_reduction/01_X_test.npy',X_test_scaled)
np.save('feature_reduction/01_y_test.npy',y_test)

# Remove 50% of features

In [ ]:
remove = importance_df.tail(6556)['Feature']
adata = adata[:, ~adata.var_names.isin(remove)]
X = adata.X
if not isinstance(X, np.ndarray):
    X_array = X.toarray()
else:
    X_array = X
y = adata.obs['broad_cell_type']
df = pd.DataFrame({'label': y.values})
np.save('main_X_50.npy', X_array)
np.save('main_Y_50.npy', y)
df.to_csv('main_df_50.csv')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_dense = X_train.toarray()
X_test_dense = X_test.toarray()
row_sums_train = X_train_dense.sum(axis=1, keepdims=True)
row_sums_test = X_test_dense.sum(axis=1, keepdims=True)
X_train_normalized = X_train_dense / row_sums_train * 1000
X_test_normalized = X_test_dense / row_sums_test * 1000
X_train_log1p = np.log1p(X_train_normalized)
X_test_log1p = np.log1p(X_test_normalized)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_log1p)
X_test_scaled = scaler.fit_transform(X_test_log1p)
np.save('feature_reduction/01_X_train.npy',X_train_scaled)
np.save('feature_reduction/01_y_train.npy',y_train)
np.save('feature_reduction/01_X_test.npy',X_test_scaled)
np.save('feature_reduction/01_y_test.npy',y_test)

# 05 - Remove 75% of the features

In [ ]:
remove = importance_df.tail(9834)['Feature']
adata = adata[:, ~adata.var_names.isin(remove)]
X = adata.X
if not isinstance(X, np.ndarray):
    X_array = X.toarray()
else:
    X_array = X
y = adata.obs['broad_cell_type']
df = pd.DataFrame({'label': y.values})
np.save('main_X_50.npy', X_array)
np.save('main_Y_50.npy', y)
df.to_csv('main_df_50.csv')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_dense = X_train.toarray()
X_test_dense = X_test.toarray()
row_sums_train = X_train_dense.sum(axis=1, keepdims=True)
row_sums_test = X_test_dense.sum(axis=1, keepdims=True)
X_train_normalized = X_train_dense / row_sums_train * 1000
X_test_normalized = X_test_dense / row_sums_test * 1000
X_train_log1p = np.log1p(X_train_normalized)
X_test_log1p = np.log1p(X_test_normalized)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_log1p)
X_test_scaled = scaler.fit_transform(X_test_log1p)
np.save('feature_reduction/01_X_train.npy',X_train_scaled)
np.save('feature_reduction/01_y_train.npy',y_train)
np.save('feature_reduction/01_X_test.npy',X_test_scaled)
np.save('feature_reduction/01_y_test.npy',y_test)

# Remove 82,5% of features

In [ ]:
remove = importance_df.tail(11473)['Feature']
adata = adata[:, ~adata.var_names.isin(remove)]
X = adata.X
if not isinstance(X, np.ndarray):
    X_array = X.toarray()
else:
    X_array = X
y = adata.obs['broad_cell_type']
df = pd.DataFrame({'label': y.values})
np.save('main_X_50.npy', X_array)
np.save('main_Y_50.npy', y)
df.to_csv('main_df_50.csv')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_dense = X_train.toarray()
X_test_dense = X_test.toarray()
row_sums_train = X_train_dense.sum(axis=1, keepdims=True)
row_sums_test = X_test_dense.sum(axis=1, keepdims=True)
X_train_normalized = X_train_dense / row_sums_train * 1000
X_test_normalized = X_test_dense / row_sums_test * 1000
X_train_log1p = np.log1p(X_train_normalized)
X_test_log1p = np.log1p(X_test_normalized)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_log1p)
X_test_scaled = scaler.fit_transform(X_test_log1p)
np.save('feature_reduction/01_X_train.npy',X_train_scaled)
np.save('feature_reduction/01_y_train.npy',y_train)
np.save('feature_reduction/01_X_test.npy',X_test_scaled)
np.save('feature_reduction/01_y_test.npy',y_test)

# Remove 93.75 of features

In [ ]:
remove = importance_df.tail(12292)['Feature']
adata = adata[:, ~adata.var_names.isin(remove)]
X = adata.X
if not isinstance(X, np.ndarray):
    X_array = X.toarray()
else:
    X_array = X
y = adata.obs['broad_cell_type']
df = pd.DataFrame({'label': y.values})
np.save('main_X_50.npy', X_array)
np.save('main_Y_50.npy', y)
df.to_csv('main_df_50.csv')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_dense = X_train.toarray()
X_test_dense = X_test.toarray()
row_sums_train = X_train_dense.sum(axis=1, keepdims=True)
row_sums_test = X_test_dense.sum(axis=1, keepdims=True)
X_train_normalized = X_train_dense / row_sums_train * 1000
X_test_normalized = X_test_dense / row_sums_test * 1000
X_train_log1p = np.log1p(X_train_normalized)
X_test_log1p = np.log1p(X_test_normalized)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_log1p)
X_test_scaled = scaler.fit_transform(X_test_log1p)
np.save('feature_reduction/01_X_train.npy',X_train_scaled)
np.save('feature_reduction/01_y_train.npy',y_train)
np.save('feature_reduction/01_X_test.npy',X_test_scaled)
np.save('feature_reduction/01_y_test.npy',y_test)

# Remove 96.8% of features

In [ ]:
remove = importance_df.tail(12702)['Feature']
adata = adata[:, ~adata.var_names.isin(remove)]
X = adata.X
if not isinstance(X, np.ndarray):
    X_array = X.toarray()
else:
    X_array = X
y = adata.obs['broad_cell_type']
df = pd.DataFrame({'label': y.values})
np.save('main_X_50.npy', X_array)
np.save('main_Y_50.npy', y)
df.to_csv('main_df_50.csv')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_dense = X_train.toarray()
X_test_dense = X_test.toarray()
row_sums_train = X_train_dense.sum(axis=1, keepdims=True)
row_sums_test = X_test_dense.sum(axis=1, keepdims=True)
X_train_normalized = X_train_dense / row_sums_train * 1000
X_test_normalized = X_test_dense / row_sums_test * 1000
X_train_log1p = np.log1p(X_train_normalized)
X_test_log1p = np.log1p(X_test_normalized)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_log1p)
X_test_scaled = scaler.fit_transform(X_test_log1p)
np.save('feature_reduction/01_X_train.npy',X_train_scaled)
np.save('feature_reduction/01_y_train.npy',y_train)
np.save('feature_reduction/01_X_test.npy',X_test_scaled)
np.save('feature_reduction/01_y_test.npy',y_test)

# Remove 98.4% of features

In [ ]:
remove = importance_df.tail(12907)['Feature']
adata = adata[:, ~adata.var_names.isin(remove)]
X = adata.X
if not isinstance(X, np.ndarray):
    X_array = X.toarray()
else:
    X_array = X
y = adata.obs['broad_cell_type']
df = pd.DataFrame({'label': y.values})
np.save('main_X_50.npy', X_array)
np.save('main_Y_50.npy', y)
df.to_csv('main_df_50.csv')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_dense = X_train.toarray()
X_test_dense = X_test.toarray()
row_sums_train = X_train_dense.sum(axis=1, keepdims=True)
row_sums_test = X_test_dense.sum(axis=1, keepdims=True)
X_train_normalized = X_train_dense / row_sums_train * 1000
X_test_normalized = X_test_dense / row_sums_test * 1000
X_train_log1p = np.log1p(X_train_normalized)
X_test_log1p = np.log1p(X_test_normalized)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_log1p)
X_test_scaled = scaler.fit_transform(X_test_log1p)
np.save('feature_reduction/01_X_train.npy',X_train_scaled)
np.save('feature_reduction/01_y_train.npy',y_train)
np.save('feature_reduction/01_X_test.npy',X_test_scaled)
np.save('feature_reduction/01_y_test.npy',y_test)

# Remove 98,8% of features

In [ ]:
remove = importance_df.tail(13010)['Feature']
adata = adata[:, ~adata.var_names.isin(remove)]
X = adata.X
if not isinstance(X, np.ndarray):
    X_array = X.toarray()
else:
    X_array = X
y = adata.obs['broad_cell_type']
df = pd.DataFrame({'label': y.values})
np.save('main_X_50.npy', X_array)
np.save('main_Y_50.npy', y)
df.to_csv('main_df_50.csv')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_dense = X_train.toarray()
X_test_dense = X_test.toarray()
row_sums_train = X_train_dense.sum(axis=1, keepdims=True)
row_sums_test = X_test_dense.sum(axis=1, keepdims=True)
X_train_normalized = X_train_dense / row_sums_train * 1000
X_test_normalized = X_test_dense / row_sums_test * 1000
X_train_log1p = np.log1p(X_train_normalized)
X_test_log1p = np.log1p(X_test_normalized)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_log1p)
X_test_scaled = scaler.fit_transform(X_test_log1p)
np.save('feature_reduction/01_X_train.npy',X_train_scaled)
np.save('feature_reduction/01_y_train.npy',y_train)
np.save('feature_reduction/01_X_test.npy',X_test_scaled)
np.save('feature_reduction/01_y_test.npy',y_test)

# Remove 99.6% of features

In [ ]:
remove = importance_df.tail(13061)['Feature']
adata = adata[:, ~adata.var_names.isin(remove)]
X = adata.X
if not isinstance(X, np.ndarray):
    X_array = X.toarray()
else:
    X_array = X
y = adata.obs['broad_cell_type']
df = pd.DataFrame({'label': y.values})
np.save('main_X_50.npy', X_array)
np.save('main_Y_50.npy', y)
df.to_csv('main_df_50.csv')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_dense = X_train.toarray()
X_test_dense = X_test.toarray()
row_sums_train = X_train_dense.sum(axis=1, keepdims=True)
row_sums_test = X_test_dense.sum(axis=1, keepdims=True)
X_train_normalized = X_train_dense / row_sums_train * 1000
X_test_normalized = X_test_dense / row_sums_test * 1000
X_train_log1p = np.log1p(X_train_normalized)
X_test_log1p = np.log1p(X_test_normalized)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_log1p)
X_test_scaled = scaler.fit_transform(X_test_log1p)
np.save('feature_reduction/01_X_train.npy',X_train_scaled)
np.save('feature_reduction/01_y_train.npy',y_train)
np.save('feature_reduction/01_X_test.npy',X_test_scaled)
np.save('feature_reduction/01_y_test.npy',y_test)

# Remove 99.8% of features

In [ ]:
remove = importance_df.tail(13087)['Feature']
adata = adata[:, ~adata.var_names.isin(remove)]
X = adata.X
if not isinstance(X, np.ndarray):
    X_array = X.toarray()
else:
    X_array = X
y = adata.obs['broad_cell_type']
df = pd.DataFrame({'label': y.values})
np.save('main_X_50.npy', X_array)
np.save('main_Y_50.npy', y)
df.to_csv('main_df_50.csv')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_dense = X_train.toarray()
X_test_dense = X_test.toarray()
row_sums_train = X_train_dense.sum(axis=1, keepdims=True)
row_sums_test = X_test_dense.sum(axis=1, keepdims=True)
X_train_normalized = X_train_dense / row_sums_train * 1000
X_test_normalized = X_test_dense / row_sums_test * 1000
X_train_log1p = np.log1p(X_train_normalized)
X_test_log1p = np.log1p(X_test_normalized)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_log1p)
X_test_scaled = scaler.fit_transform(X_test_log1p)
np.save('feature_reduction/01_X_train.npy',X_train_scaled)
np.save('feature_reduction/01_y_train.npy',y_train)
np.save('feature_reduction/01_X_test.npy',X_test_scaled)
np.save('feature_reduction/01_y_test.npy',y_test)

# Remove 99,9% of features

In [ ]:
remove = importance_df.tail(13100)['Feature']
adata = adata[:, ~adata.var_names.isin(remove)]
X = adata.X
if not isinstance(X, np.ndarray):
    X_array = X.toarray()
else:
    X_array = X
y = adata.obs['broad_cell_type']
df = pd.DataFrame({'label': y.values})
np.save('main_X_50.npy', X_array)
np.save('main_Y_50.npy', y)
df.to_csv('main_df_50.csv')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_dense = X_train.toarray()
X_test_dense = X_test.toarray()
row_sums_train = X_train_dense.sum(axis=1, keepdims=True)
row_sums_test = X_test_dense.sum(axis=1, keepdims=True)
X_train_normalized = X_train_dense / row_sums_train * 1000
X_test_normalized = X_test_dense / row_sums_test * 1000
X_train_log1p = np.log1p(X_train_normalized)
X_test_log1p = np.log1p(X_test_normalized)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_log1p)
X_test_scaled = scaler.fit_transform(X_test_log1p)
np.save('feature_reduction/01_X_train.npy',X_train_scaled)
np.save('feature_reduction/01_y_train.npy',y_train)
np.save('feature_reduction/01_X_test.npy',X_test_scaled)
np.save('feature_reduction/01_y_test.npy',y_test)

# Remove 99.95% of features

In [ ]:
remove = importance_df.tail(13106)['Feature']
adata = adata[:, ~adata.var_names.isin(remove)]
X = adata.X
if not isinstance(X, np.ndarray):
    X_array = X.toarray()
else:
    X_array = X
y = adata.obs['broad_cell_type']
df = pd.DataFrame({'label': y.values})
np.save('main_X_50.npy', X_array)
np.save('main_Y_50.npy', y)
df.to_csv('main_df_50.csv')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_dense = X_train.toarray()
X_test_dense = X_test.toarray()
row_sums_train = X_train_dense.sum(axis=1, keepdims=True)
row_sums_test = X_test_dense.sum(axis=1, keepdims=True)
X_train_normalized = X_train_dense / row_sums_train * 1000
X_test_normalized = X_test_dense / row_sums_test * 1000
X_train_log1p = np.log1p(X_train_normalized)
X_test_log1p = np.log1p(X_test_normalized)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_log1p)
X_test_scaled = scaler.fit_transform(X_test_log1p)
np.save('feature_reduction/01_X_train.npy',X_train_scaled)
np.save('feature_reduction/01_y_train.npy',y_train)
np.save('feature_reduction/01_X_test.npy',X_test_scaled)
np.save('feature_reduction/01_y_test.npy',y_test)

In [ ]:
len(adata.var)

In [ ]:
sc.pl.umap(adatab, color='TMEM144') #Expressed by oligo in their differentiation from OPC

In [ ]:
sc.pl.umap(adatab, color='CKB') #Possível novo marcador de glioblastoma

In [ ]:
sc.pl.umap(adatab, color='CSF1R') #Marcador de macrófagos

In [ ]:
sc.pl.umap(adatab, color='CD34') #Marcador de células endoteliais

In [ ]:
sc.pl.umap(adatab, color='QDPR')

In [ ]:
sc.pl.umap(adatab, color='IL2RG')

In [ ]:
sc.pl.umap(adatab, color='CNDP1')

In [ ]:
sc.pl.umap(adatab, color='SRGN')

In [ ]:
sc.pl.umap(adatab, color='CD96')

In [ ]:
adata.var.head(10)